### Unit testing for MPI

In [1]:
import camber
import os
import time
from IPython.utils import io
import numpy as np

In [ ]:
# SWD -- cannot currently change working dir as camber.mpi.run puts all output it top level directory
#os.chdir("/pathto/mpitests")
#os.getcwd()

In [23]:
#Copy input file from inputs directory to working directory
os.system("cp mpi_inputs/athinput.orszag-tang-test .")

0

In [3]:
# Using python timing for performance evaluation, would be better/useful to also get diagnostics directly from Athena++ output
worker_size = "LARGE"
ti = time.time()
worker = camber.mpi.run(
    command="mpirun -np 32 athena/bin/athena -i athinput.orszag-tang-test",
    worker_size=worker_size
)
tf = time.time()
wait_start_time = tf-ti

Output()

In [21]:
# get diagnostics from logs
with io.capture_output() as captured:
    worker.read_logs(tail_lines=2)
walltime = float(captured.stdout.split()[6])
cyclerate = float(captured.stdout.split()[11])

In [25]:
# copy and compile files for joining vtk outputs
os.system("cp mpi_inputs/athena_read.py .")
import athena_read
os.system("cp mpi_inputs/join_vtk++.c .")
os.system("gcc -o join_vtk++ join_vtk++.c")

0

In [34]:
# create joined file from last output
os.system("cp ../OrszagTang.block*.out2.00100.vtk .")
os.system("./join_vtk++ -o final.vtk OrszagTang.*.out2.00100.vtk")

0

In [37]:
# Check that output matches target
x, y, z, final = athena_read.vtk("final.vtk")
x, y, z, target = athena_read.vtk("mpi_inputs/target.vtk")
error = np.average(abs(final['rho']-target['rho'])/target['rho'])

In [41]:
print("MPI start-uptime this run: {:e} for worker size ".format(wait_start_time)+worker_size)
print("Orszag-Tang time this run: {:e} for worker size ".format(walltime)+worker_size)
print("Orszag-Tang zone cycles/second this run: {:e}".format(cyclerate))
print("Expected zones cycles per second is ~2e7")
print("Mean error in density in final frame: {:e}".format(error))


MPI start-uptime this run: 5.196135e+00 for worker size LARGE
Orszag-Tang time this run: 4.331959e+01 for worker size LARGE
Orszag-Tang zone cycles/second this run: 2.209970e+07
Expected zones cycles per second is ~2e7
Mean error in density in final frame: 0.000000e+00


In [44]:
# clean up files in working directory
os.system("rm athinput.orszag-tang-test")
os.system("rm athena_read.py")
os.system("rm join_vtk++.c")
os.system("rm join_vtk++")
os.system("rm OrszagTang.block*.out2.00100.vtk final.vtk")

256